# Parse BioContainers containers to dico

https://github.com/BioContainers/containers

In [19]:
import os
import glob
import re
import json

In [20]:
download = True

In [21]:
if(download):
    os.system("git clone https://github.com/BioContainers/containers.git")

Clonage dans 'containers'...


In [22]:
dico = {}

index = 0
for file in glob.glob(f'./containers/**/Dockerfile', recursive=True):
    with open(file) as f:
        content = f.read()
        tool=""
        for match in re.finditer(r'software *= *"([^\"\/]+)"', content):
            tool = match.group(1).lower()
        #Create tool entry in the dictionnary -> if not already created
        try: 
            a = dico[tool]
        except:
            dico[tool] = {"tags" : [], "summary" : []}

        for match in re.finditer(r'tags *= *"([^\"\/]*)"', content):
            dico[tool]["tags"] += match.group(1).split(",")

        for match in re.finditer(r'summary *= *"([^\"\/]*)"', content):
            dico[tool]["summary"].append(match.group(1))

for tool in dico:
    for i in range(len(dico[tool]["tags"])):
        #TODO -> might need to rework the tags in the futur (since the tags themselves are structured)
        dico[tool]["tags"][i] = dico[tool]["tags"][i].strip()
    #We remove the doubles
    dico[tool]["tags"] = list(set(dico[tool]["tags"]))
    dico[tool]["summary"] = list(set(dico[tool]["summary"]))

In [23]:
with open("biocontainers.json", "w") as outfile:
    json.dump(dico, outfile, indent = 4)

In [25]:
if(download):
    os.system("rm -rf containers/")